## 示例1

https://github.com/huggingface/transformers/issues/7336

In [ ]:
from transformers import (BertForNextSentencePrediction,
                          BertTokenizer,
                          RobertaModel, RobertaTokenizer, Trainer,
                          TrainingArguments)
from transformers.data.datasets.language_modeling import TextDatasetForNextSentencePrediction
from transformers.data.data_collator import DataCollatorForNextSentencePrediction
from argparse import ArgumentParser


def parse_args():
    parser = ArgumentParser("Fine-tune RoBERTa in Next Sentence Prediction.")
    parser.add_argument("-m", "--model_path", dest="model_path", required=True, help="Path to RoBERTa model.")
    parser.add_argument("-o", "--output_path", dest="output_path", required=True, help="Path to directory of fine-tuned model.")
    parser.add_argument("-d", "--dataset_path", dest="dataset_path", required=True, help="Path to dataset.")
    args = parser.parse_args()
    return args


if __name__ == "__main__":
    args = parse_args()
    tokenizer = RobertaTokenizer.from_pretrained(args.model_path)
    finetune_model = BertForNextSentencePrediction.from_pretrained(args.model_path)

    training_args = TrainingArguments(
        output_dir=args.output_path,
        num_train_epochs=3,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
    )

    data_collator = DataCollatorForNextSentencePrediction(
        tokenizer=tokenizer,
        mlm=False,
        block_size=512,
        nsp_probability=0.5,
      )

    train_dataset = TextDatasetForNextSentencePrediction(
        tokenizer=tokenizer,
        file_path=args.dataset_path,
        block_size=512,
    )

    trainer = Trainer(
        model=finetune_model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=data_collator,
    )

    trainer.train()
    trainer.save_model(args.output_path)

## 示例2 BERT-SCRATCH.ipynb
- https://discuss.huggingface.co/t/training-bert-from-scratch-mlm-nsp-on-a-new-domain/2075
- https://colab.research.google.com/drive/12NHfXeUBo7RBl3Kffa-715i-Zpd0MOzP?usp=sharing

## 示例3 mlm-and-nsp

https://stackoverflow.com/questions/65646925/how-to-train-bert-from-scratch-on-a-new-domain-for-both-mlm-and-nsp